In [1]:
from datasets import load_dataset
ds = load_dataset('eth-dl-rewards/pref-data-math', split='eval')

In [4]:
ds.num_rows

13544

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

MODEL = 'eth-dl-rewards/internlm2-7b-reward-math-30k'
model = AutoModel.from_pretrained(MODEL, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='cuda')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

In [4]:
def get_reward(prompt, solution):
  messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": solution}
  ]
  encoded = tokenizer.apply_chat_template(messages, tokenize=False)
  with torch.no_grad():
    inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=3500).to('cuda')
    print(inputs['input_ids'].shape)
    outputs = model(**inputs)
    reward = outputs.logits[0][0].item()
    del inputs
    del outputs
    gc.collect()
    torch.cuda.empty_cache()

    return reward


In [ ]:
from tqdm import tqdm
import gc
correct = 0
total = 0
gaps = []
model.eval()
for problem, accepted, rejected in tqdm(zip(ds['problem'], ds['accepted'], ds['rejected']), total=len(ds), desc='Evaluating'):
  reward_accepted = get_reward(problem, accepted)
  reward_rejected = get_reward(problem, rejected)
  gaps.append(reward_accepted - reward_rejected)
  if reward_accepted > reward_rejected:
    correct += 1
  total += 1

  if total % 100 == 0:
    print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")
    gc.collect()
    torch.cuda.empty_cache()

Evaluating:   0%|          | 0/13544 [00:00<?, ?it/s]

torch.Size([1, 648])
torch.Size([1, 723])


Evaluating:   0%|          | 1/13544 [00:00<2:44:38,  1.37it/s]

torch.Size([1, 648])
torch.Size([1, 1395])


Evaluating:   0%|          | 2/13544 [00:01<2:34:35,  1.46it/s]

...

Evaluating:   2%|▏         | 320/13544 [02:53<1:26:13,  2.56it/s]

torch.Size([1, 204])
torch.Size([1, 189])


Evaluating:   2%|▏         | 321/13544 [02:53<1:24:40,  2.60it/s]

torch.Size([1, 218])
torch.Size([1, 189])


Evaluating:   2%|▏         | 322/13544 [02:53<1:21:39,  2.70it/s]

In [6]:
print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")

Correct: 4062/4180 (97.18%)


In [2]:
from tqdm import tqdm
import numpy as np
import gc
import torch

def get_rewards_batch(batch):
    prompts, solutions = zip(*batch)
    messages = [
        [{"role": "user", "content": prompt}, {"role": "assistant", "content": solution}]
        for prompt, solution in zip(prompts, solutions)
    ]
    encoded = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=4096, padding=True).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
        rewards = outputs.logits[:, 0].tolist()
        del inputs, outputs
        return rewards

def precompute_rewards(ds, batch_size=10):
    rewards = []
    for i in tqdm(range(0, len(ds), batch_size), desc='Precomputing Rewards'):
        batch = list(zip(ds['problem'][i:i + batch_size], ds['accepted'][i:i + batch_size], ds['rejected'][i:i + batch_size]))
        accepted_batch = [(problem, accepted) for problem, accepted, _ in batch]
        rejected_batch = [(problem, rejected) for problem, _, rejected in batch]

        reward_accepted = get_rewards_batch(accepted_batch)
        reward_rejected = get_rewards_batch(rejected_batch)

        batch_rewards = list(zip(reward_accepted, reward_rejected))
        rewards.extend(batch_rewards)

        gc.collect()
        torch.cuda.empty_cache()

    return rewards

def evaluate_from_precomputed(rewards):
    correct = 0
    total = 0
    gaps = []

    for reward_accepted, reward_rejected in rewards:
        gap = reward_accepted - reward_rejected
        gaps.append(gap)
        if reward_accepted > reward_rejected:
            correct += 1
        total += 1

    return correct, total, gaps

# Example usage:
# ds should be a list of tuples in the form [(problem, accepted, rejected), ...]
batch_size = 8
precomputed_rewards = precompute_rewards(ds, batch_size=batch_size)
correct, total, gaps = evaluate_from_precomputed(precomputed_rewards)
print(f"Final Results: Correct: {correct}/{total} ({correct/total*100:.2f}%)")


NameError: name 'ds' is not defined

In [5]:
from transformers import AutoConfig, AutoModel
config = AutoConfig.from_pretrained("internlm/internlm2-7b")

config.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

configuration_internlm2.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [10]:
config.auto_map['AutoModel'] = 'internlm/internlm2-7b--modeling_internlm2.InternLM2ForSequenceClassification'

'internlm/internlm2-7b--modeling_internlm2.InternLM2ForCausalLM'